In [ ]:
%matplotlib inline
import sys, os
import numpy as np
import inspect
from openxrd import find_peaks_2d
from openxrd import find_peaks_1d
from openxrd import find_saddle_1d
from openxrd import fit_single
from openxrd import get_fit_all_1d
from openxrd import get_fit_all_2d
from openxrd import fits_to_csv_multitype
from openxrd import fit_data_to_csv
from openxrd import BrukerData
from openxrd import fit_multipeak
from openxrd import fits_to_csv
from openxrd import csv_append_col

from datasetmetta import get_name_data

import matplotlib.pyplot as plt
from matplotlib.ticker import LinearLocator, FormatStrFormatter
import matplotlib.patheffects as path_effects
from mpl_toolkits.axes_grid1 import make_axes_locatable

import csv
from lmfit import models
from lmfit import lineshapes
from scipy.signal import savgol_filter
from scipy.signal import find_peaks_cwt
from scipy import signal
import scipy

from scipy.signal import argrelextrema
#PLOT Setup

axlabels = {'family':               'sans',
            'color':                'black',
            'weight':               'normal',
            'size':                 20,
            }
titles = {'family':                 'sans',
          'color':                  'black',
          'weight':                 'normal',
          'size':                   24,
          }
labels = {'family':                 'sans',
          'fontname':               'DejaVu Sans',
          'color':                  '#000000', #'#66ff33',
          'weight':                 'normal',
          'size':                   14,
          'verticalalignment':      'center',
          'horizontalalignment':    'right'
          }
ledg =   {'size':                    14,
          'family':                  'sans'
         }
def butter_lp_filter(y, cutoff=None, order=2):
    """
    Performs a low pass butterworth filter on the data with cutoff
    frequency that defaults to 2/len(y)
    Arguments
    ---------
    cutoff (float) [default: 2/len(y)]
        cutoff frequency at which to filter the data
    order (int) [default: 2]
        filter order
    Returns
    -------
    (array)
        low pass filtered array same size as data
        
    """
    
    if cutoff is None:
        cutoff = 2 / len(y)
    B, A = signal.butter(order, cutoff, output='ba')
    return signal.filtfilt(B, A, y)

def butter_hp_filter(y, cutoff=None, order=2):
    
    if cutoff is None:
        cutoff = 2 / len(y)
    B, A = signal.butter(order, cutoff, btype='high')
    return signal.filtfilt(B, A, y)

def find_peaks(y, y_r, y_sd, data, sigma=3, neigh_multi=0.01, neigh_add=5):
    neighbors = int(y.size * neigh_multi) + neigh_add
    (y_mx,) = argrelextrema(y, np.greater_equal, order=neighbors, mode='clip')
    #print(y_mx)
    (peaks,_) = data.get_real_xy(y_mx.tolist())
    #print('peaks', peaks)
    #for i, p in enumerate(y_mx):
    #    if y[p] > y_r[p] +sigma*y_sd[p]:
    #        print(peaks[i], y_r[p], y_sd[p], y[p], y_r[p] + sigma* y_sd[p])
            
    y_mx = [p for p in y_mx if y[p] > y_r[p] + sigma* y_sd[p]]
    (peaks,_) = data.get_real_xy(y_mx)
    #print(peaks)
    return peaks
    #(peaks,_) = data.get_real_xy(peaks)
    #peaks = [round(r) for r in peaks]
    
    
        #def find_peaks(ax, x, y, data=None, neigh=None, threshold=1000, peak_widths=(10, 25)):
        #print(len(y), max(x))
        #peak_widths = (len(y)/max(x)/2, len(y)/max(x)*10)
        #print(peak_widths)
        #get peaks in data
        #peaks = find_peaks_cwt(y, neigh)
        #if neigh:
        #    data_max = np.asarray(find_peaks_1d(y, neigh[0], neigh[1]))
        #    data_min = np.asarray(find_saddle_1d(y, neigh[0], neigh[1]))
        #else:
        #    data_max = np.asarray(find_peaks_1d(y))
        #    data_min = np.asarray(find_saddle_1d(y))
        #maxima = (y == data_max)
        #diff = ((data_max - data_min) > threshold)
        #maxima[diff == 0] = 0
        #peaks = [m for m in x if maxima[m]]
        #peaks = (signal.find_peaks_cwt(y, peak_widths)).tolist()
        #print(peaks)
        #(peaks,_) = data.get_real_xy(peaks)
        #peaks = [round(r) for r in peaks]
        #print(peaks)

def get_pdf(pdffile, peaks, ax, x, y):
        # fastest?
        # https://softwarerecs.stackexchange.com/questions/7463/fastest-python-library-to-read-a-csv-file
        with open(pdffile, 'r') as f:
            pdfs = csv.reader(row for row in f if not
                               row.startswith('#'))
            pdfs = np.asarray([p for p in pdfs])
        x_p = [float(p[0]) for p in pdfs]
        print(peaks)
        for pk in peaks:
            match = np.abs(x_p-pk).argmin()
            j = float(pdfs[match][0])
            k = float(pdfs[match][1])
            txt = pdfs[match][2]
            color='black'
            if abs(j-pk)>0.6:
                j = pk
                k = float(100)
                txt = '??'
                color='red'
            amp = y.max() * k / 100.0
                    
            txt = ax.text(j, 0.1, txt,
                          fontdict=labels, rotation=90)

            line = ax.plot([j, j],[0.01, amp], 
                           linestyle=':', color=color)

def smooth(y, box_pts, poly=None):
    if poly:
        y_smooth = savgol_filter(y, box_pts, poly)
    else:
        box = np.ones(box_pts)/box_pts
        y_smooth = np.convolve(y, box, mode='same')
    return y_smooth


def get_background(x, y, neigh_multi=0.01, neigh_add=0, cutper = 2):
    neighbors = int(y.size * neigh_multi) + neigh_add
    y_r = np.copy(y)
    y_sd = np.zeros(len(y))
    i = 0
    while i+2*neighbors < len(y):
        mean1 = np.mean(y_r[i:i+neighbors])
        sd1 = y_sd[i]
        #print(sd1)
        if sd1 == 0:
            sd1 = np.std(y[i:i+neighbors])
            y_sd[i:i+neighbors] = sd1
        # removing outlires > 2std     
        y_r[i:i+neighbors] = [c if c < mean1+2*sd1 else mean1 for c in y_r[i:i+neighbors]]
        
        mean2 = np.mean(y_r[i+neighbors:i+2*neighbors])
        sd2 = np.std(y_r[i+neighbors:i+2*neighbors])
        #print(sd1, sd2)
        if mean2 > cutper*mean1 or sd2 > cutper*sd1:
            y_r[i+neighbors:i+2*neighbors] = mean1
            y_sd[i+neighbors:i+2*neighbors] = sd1
        else:
            y_sd[i+neighbors:i+2*neighbors] = sd2
        i += neighbors

    return y_r, y_sd

def main(files, pdffile, name=None, peak_add=None, peak_sub=None, label_type='position'):
    fig = plt.figure(figsize=(10, 6))#, dpi=100)
    ax = fig.add_subplot(111)
    ax.set_title(name, fontdict=titles, y=1)
    #axis Lables
    ax.set_xlabel('2\u03b8[\u00b0]', fontdict=titles)
    # ax.set_xlabel('\u03c9[\u00b0]', fontdict=titles) #RC
    ax.set_ylabel(u'Intensity [arbitrary units]', fontdict=titles)
    # Set axis tick labels font
    for label in (ax.get_xticklabels() + ax.get_yticklabels()):
        for prop in axlabels:
            getattr(label, 'set_' + prop)(axlabels[prop])
    for ii, file in enumerate(files):
        data = BrukerData(file)
        #print(len(data.rngs))
        for j, rng in enumerate(data.rngs):
            #print (j)
            twoth_0 = rng.metta['start_2th']
            twoth_s = rng.metta['step_size']
            twoth_l = rng.metta['steps']
            twoth_e = twoth_0 + twoth_s*twoth_l
            x = np.linspace(twoth_0, twoth_e, twoth_l) 
            if j == 0:
                ax.set_xlim([min(x), max(x)])
            y = np.asarray(rng.counts_data)
            #print(len(y))
            y[y< 1] = 1
            d_x = rng.metta['drive_x']
            d_y = rng.metta['drive_y']
            d_r = rng.metta['drive_phi']
            if label_type == 'position':
                ax.semilogy(x, y, label='({0:.0f},{1:.0f}) {2:.0f}\u00b0'.format(d_x, d_y, d_r)) #plot
            else:
                ax.semilogy(x, y, label='{0}'.format(label_type[ii])) #plot    
            ax.set_ylim(bottom=0.01, top=max(y)*10)
            ax.set_yticklabels([])
            # plot smoothed if using
            #if smo:
            #    y = smooth(y, smo[0], smo[1])
            #    ax.plot(x, y) # window size 51, polynomial order 3
            if j == 0:    
                # Fit background with lmfit and compair to background to determine if true peak    
                # first remove all points above the average
                y_r, y_sd = get_background(x, y)
                #ax.plot(x, y_r, 'r-')
            
                # Plot StD
                #ax2 = ax.twinx()
                #ax2. plot(x, y_sd, 'g-')
                # smooth (fit) StD
                mod = models.PolynomialModel(3)
                pars = mod.guess(y_sd, x=x)
                out = mod.fit(y_sd, pars, x=x)
                y_sd = out.best_fit
                # Plot SStd
                #ax2. plot(x, y_sd, 'g:')
            
                # find background function
                mod = models.PolynomialModel(3)
                pars = mod.guess(y_r, x=x)
                out = mod.fit(y_r, pars, x=x)
                y_r = out.best_fit
                # Plot Background function
                #ax.plot(x, y_r, 'r-')
            
            
                peaks = find_peaks(y, y_r, y_sd, data, sigma=4)
                if peak_sub is not None:
                    for pk in peak_sub:
                         del peaks[np.abs(np.asarray(peaks)-pk).argmin()]
                if peak_add is not None:
                    peaks = np.append(peaks, peak_add)


                #ax.plot(x, y_r, 'b:')
                get_pdf(pdffile, peaks, ax, x, y)

            # Maybe look more into this in the future...
            # ax.plot(x, scipy.fftpack.fft(y), 'g-')

            #ax.plot(x, butter_lp_filter(y,0.1,3))
            #ax.plot(x, butter_hp_filter(butter_lp_filter(y,0.01,3),.0000019,3))
            
            #get_pdf(pdffile, ax, x, y, data, neigh)
            
    plt.legend(#bbox_to_anchor=(1., 1.02, 1., .102),
           loc= 9, ncol=5, mode="expand", borderaxespad=0., prop=ledg)        
    plt.tight_layout()
    plt.show()


# Get data
root = '/mnt/W/Austin_Fox/'
rootf = root + 'XRD/data/'

In [ ]:
name = '8721'
files = [rootf + name +'.raw']
pdffile = root + 'devel/BAW_peaks.csv'
y =main(files, pdffile, name, peak_sub=[56.8])

In [ ]:
name = '8721'
name2 = '8722'
print(name)
files = [rootf + name +'.raw', rootf + name2 +'.raw']
pdffile = root + 'devel/BAW_peaks.csv'
main(files, pdffile, name + " - " + name2, label_type=[name, name2])

In [ ]:
name = '7050'
print(name)
files = [rootf + name +'.raw']
pdffile = root + 'devel/BAW_peaks.csv'
main(files, pdffile, name)

In [ ]:
name = '6439'
print(name)
files = [rootf + name +'.raw']
pdffile = root + 'devel/BAW_peaks.csv'
main(files, pdffile, name, peak_sub=[40.9, 72.8, 74.3])

In [ ]:
name = '7527'
print(name)
files = [rootf + name +'.raw']
pdffile = root + 'devel/BAW_peaks.csv'
main(files, pdffile, name, peak_sub=[89.9, 88.8, 55.6, 26.1, 53.2, 83.8], peak_add=[25.5, 52.5, 83.4])

In [ ]:
name = '8718_full'
print(name)
files = [rootf + name +'.raw']
pdffile = root + 'devel/BAW_peaks.csv'
main(files, pdffile, '8718', peak_add=[57.7, 77.9, 38.4])

In [ ]:
name = '8720_fastmap'
print(name)
files = [rootf + name +'.raw']
pdffile = root + 'devel/BAW_peaks.csv'
main(files, pdffile, '8720', peak_add=[57.7, 77.9, 38.4])

In [ ]:
name = '8723_full_map'
print(name)
files = [rootf + name +'.raw']
pdffile = root + 'devel/BAW_peaks.csv'
main(files, pdffile, '8723', peak_add=[57.7, 77.9, 38.4])

In [ ]:
name = 'S2_P1_Al'
print(name)
files = [rootf + name +'.raw']
pdffile = root + 'devel/BAW_peaks.csv'
main(files, pdffile, name, peak_sub=[40.1, 42, 74.7, 69.14, 32.97], peak_add=[69.13, 32.96])

In [ ]:
name = 'S2_P2_Si'
print(name)
files = [rootf + name +'.raw']
pdffile = root + 'devel/BAW_peaks.csv'
main(files, pdffile, name, peak_sub=[74.0, 41, 43.5, 69.14], peak_add=[69.13, 32.96])

In [ ]:
name = 'S3_P3_Al'
print(name)
files = [rootf + name +'.raw']
pdffile = root + 'devel/BAW_peaks.csv'
main(files, pdffile, name, peak_sub=[41, 43.5, 69.14, 33, 33], peak_add=[69.13, 32.96])

In [ ]:
name = 'S3_P4_TOx'
print(name)
files = [rootf + name +'.raw']
pdffile = root + 'devel/BAW_peaks.csv'
main(files, pdffile, name, peak_sub=[22.6, 69.14, 32.97], peak_add=[69.13, 32.96])

In [ ]:
name = '8720_fastmap'
print(name)
files = [rootf + name +'.raw']
pdffile = root + 'devel/BAW_peaks.csv'
main(files, pdffile, name)

In [ ]:
name = 'S2_P2_Si_redge'
print(name)
files = [rootf + name +'.raw']
pdffile = root + 'devel/BAW_peaks.csv'
main(files, pdffile, 'S2_P2_Si_right-edge', peak_sub=[89,89,89,89,69], peak_add=[69])

In [ ]:
name = 'S2_P1_Al_edge'
print(name)
files = [rootf + name +'.raw']
pdffile = root + 'devel/BAW_peaks.csv'
main(files, pdffile, name, peak_sub=[42,69.14, 32.97], peak_add=[69.13, 32.96])

In [ ]:
name = 'S2_P2_Si_2th'
print(name)
files = [rootf + name +'.raw']
pdffile = root + 'devel/BAW_peaks.csv'
main(files, pdffile, 'S2_P2_Si_right-edge', peak_sub=[89,89,89,89,69], peak_add=[69])